In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from scipy.spatial import KDTree
import numpy as np

# Additional imports
import pystac_client
import xarray as xr
import pathlib
import os
import geopandas as gpd
import matplotlib as mpl
from matplotlib import colors
import rasterio

import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
from rasterio.plot import show

import matplotlib.colors as mcolors
from shapely.geometry import Point
import pandas as pd

from tqdm import tqdm

import webbrowser
from pathlib import Path
import os
import time

from itertools import compress
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from tqdm import tqdm

from shapely.geometry import box



Cannot find header.dxf (GDAL_DATA is not defined)


In [2]:
# opening STAC catalog

catalog = pystac_client.Client.open(
    #"https://storage.googleapis.com/dgds-data-public/coclico/coclico-stac/catalog.json" # cloud API
    "https://storage.googleapis.com/dgds-data-public/gca/SOTC/gca-stac_sotc/catalog.json" # P drive API 
)
#catalog

In [3]:
list(catalog.get_children())

[<CollectionClient id=shore_mon_fut>,
 <CollectionClient id=sed_class>,
 <CollectionClient id=shore_mon>,
 <CollectionClient id=world_pop>,
 <CollectionClient id=world_gdp>,
 <CollectionClient id=shore_mon_hr>,
 <CollectionClient id=shore_mon_drivers>,
 <CollectionClient id=esl_gwl>,
 <CollectionClient id=sub_threat>,
 <CollectionClient id=Exp_world_pop>,
 <CollectionClient id=Exp_world_pop_parquet>,
 <CollectionClient id=Haz-ESL>,
 <CollectionClient id=Vul-Inf_Mort>,
 <CollectionClient id=Haz-GCM_90-100>,
 <CollectionClient id=Exp-NS_Slopes>,
 <CollectionClient id=Vul_MYS>,
 <CollectionClient id=Vul_RS>,
 <CollectionClient id=Haz-GCM_40-50>,
 <CollectionClient id=Haz-ERA5_04-14>,
 <CollectionClient id=Vul-GDP_Cap>,
 <CollectionClient id=Haz-GCM_04-14>,
 <CollectionClient id=Haz-Land_Sub_2010_COGs>,
 <CollectionClient id=Haz-Land_Sub_2040_COGs>,
 <CollectionClient id=Vul-Roads_Density_COGs>,
 <CollectionClient id=Exp-Delta_DEM_COGs>]

In [4]:
href = catalog.get_child("shore_mon").assets["data"].href
ds = xr.open_zarr(href)
ds

c:\Users\rowe\AppData\Local\miniconda3\envs\coclico\Lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:             (stations: 1837203, time: 38)
Coordinates: (12/18)
    coastline_idint     (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    continent           (stations) |S23 dask.array<chunksize=(28707,), meta=np.ndarray>
    country             (stations) |S40 dask.array<chunksize=(28707,), meta=np.ndarray>
    country_id          (stations) |S7 dask.array<chunksize=(114826,), meta=np.ndarray>
    err_changerate      (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    err_timespan        (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    ...                  ...
    no_shorelines       (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    rmse                (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
  * time                (time) datetime64[ns] 1984-01-01 ... 2021-01-01
    timespan            (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    transect_geom       (stations) |S92 dask.array<chunksize=(14354,), meta=np.ndarray>
    transect_id         (stations) |S15 dask.array<chunksize=(57413,), meta=np.ndarray>
Dimensions without coordinates: stations
Data variables:
    changerate          (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    changerate_unc      (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    outliers            (stations, time) float32 dask.array<chunksize=(114826, 3), meta=np.ndarray>
    sandy               (stations) int8 dask.array<chunksize=(459301,), meta=np.ndarray>
    sp                  (stations, time) float64 dask.array<chunksize=(57413, 3), meta=np.ndarray>
Attributes: (12/21)
    AUTHOR:              Luijendijk, A., Hagenaars, G., Ranasinghe, R., Baart...
    CITATION:            Luijendijk, A., Hagenaars, G., Ranasinghe, R. et al....
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8
    DESCRIPTION:         The Shoreline Monitor is a dataset that incorporates...
    ...                  ...
    SPATIAL_EXTENT:      [-180, -90, 180, 90]
    TAGS:                ['civil engineering', 'geomorphology', 'shoreline dy...
    TEMPORAL_EXTENT:     ['1984-01-01T00:00:00Z', '2021-01-01T00:00:00Z']
    TITLE:               The Shoreline Monitor
    TITLE_ABBREVIATION:  shore_mon
    UNITS:

In [5]:
# href2='https://storage.googleapis.com/dgds-data-public/coclico/shoreline_change_projections.zarr/global_shoreline_change_projections.zarr'
# ds = xr.open_zarr(href2)
# ds

In [6]:
# ds=ds.isel(time=0)
ds=ds.compute()

In [8]:
dss=ds
ds=ds.isel(time=0)

In [9]:
df=ds.to_dataframe()
df

,changerate,changerate_unc,coastline_idint,continent,country,country_id,err_changerate,err_timespan,intercept,intercept_unc,...,no_sedcomp,no_shorelines,outliers,rmse,sandy,sp,time,timespan,transect_geom,transect_id
stations,,,,,,,,,,,,,,,,,,,,,
0,14.088576,1.503763,5389.0,b'South America',b'Chile',b'CHL',1.0,1.0,56.678813,35.137902,...,1.0,8.0,0.0,23.620801,1,NaN,1984-01-01,21.0,"b'LINESTRING (-74.3863095545 -50.3776589451, -...",b'BOX_028_183_0'
1,6.937372,7.717642,5389.0,b'South America',b'Chile',b'CHL',1.0,1.0,297.416535,196.569308,...,1.0,13.0,1.0,157.453404,0,414.190582,1984-01-01,21.0,"b'LINESTRING (-74.382468591 -50.3791437735, -7...",b'BOX_028_183_1'
2,2.430833,1.637254,5389.0,b'South America',b'Chile',b'CHL',1.0,1.0,282.545984,41.701053,...,1.0,14.0,1.0,33.402840,0,367.236267,1984-01-01,21.0,"b'LINESTRING (-74.3786276274 -50.3806285554, -...",b'BOX_028_183_2'
3,4.427241,10.685796,5389.0,b'South America',b'Chile',b'CHL',1.0,1.0,212.780267,278.138758,...,1.0,15.0,1.0,208.282579,0,484.592266,1984-01-01,21.0,b'LINESTRING (-74.3739502728 -50.3825830980000...,b'BOX_028_183_3'
4,4.002978,10.248722,5389.0,b'South America',b'Chile',b'CHL',1.0,1.0,271.142825,255.122503,...,1.0,16.0,0.0,308.598295,0,393.579932,1984-01-01,35.0,"b'LINESTRING (-74.3704249118 -50.3843577821, -...",b'BOX_028_183_4'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837198,12.993559,2.265287,17612.0,b'Europe',b'Russia',b'RUS',1.0,1.0,21.928761,45.436937,...,1.0,36.0,1.0,116.339704,0,284.891489,1984-01-01,36.0,"b'LINESTRING (39.9299368881 64.7014617708, 39....",b'BOX_211_067_149'
1837199,8.107627,2.017299,17612.0,b'Europe',b'Russia',b'RUS',1.0,1.0,196.397698,44.916515,...,1.0,36.0,1.0,99.278079,0,190.727131,1984-01-01,34.0,"b'LINESTRING (39.9335774372 64.7025859435, 39....",b'BOX_211_067_150'
1837200,1.779037,0.154548,17612.0,b'Europe',b'Russia',b'RUS',1.0,1.0,503.516226,3.161074,...,1.0,36.0,0.0,8.878198,0,-6.202073,1984-01-01,36.0,"b'LINESTRING (39.9355464131 64.7035022455, 39....",b'BOX_211_067_151'


In [16]:
df.loc[df["transect_id"]==b'BOX_138_166_6',"changerate_unc"]

stations
861038    3.553623
Name: changerate_unc, dtype: float64

In [ ]:
# import geopandas as gpd
# df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\01_Hazards\01_Present\Shoreline_Change\Global_Erosion_rate_PC.csv")
# data_points = gpd.GeoDataFrame(
#     df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
# )
# data_points

In [15]:
href

'https://storage.googleapis.com/dgds-data-public/gca/shoreline_monitor.zarr'